In [6]:
from __future__ import print_function
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import numpy as np
import os 
import glob
import skimage.io as io
import skimage.transform as trans
from tensorflow.keras.models import *
from tensorflow.keras.layers import *
from tensorflow.keras.optimizers import *
from tensorflow.keras.callbacks import ModelCheckpoint, LearningRateScheduler
from tensorflow.keras.models import *
from tensorflow.keras.layers import *
from tensorflow.keras.optimizers import *
from tensorflow.keras import backend as K
from unet_model import unet,dice_coef,dice_coef_loss
from data import adjustData,trainGenerator,testGenerator,labelVisualize,saveResult
from tensorflow.keras.callbacks import  EarlyStopping, ReduceLROnPlateau 
import matplotlib.pyplot as plt

In [7]:
import numpy as np
import cv2
import os
from PIL import Image

X_test = []
for filepath in os.listdir('../data/test_images/ventral_samples_R0004'):
    image = Image.open('../data/test_images/ventral_samples_R0004/'+filepath)
    image = image.resize((256, 256))
    # convert image to numpy array
    data = np.asarray(image)
    data = data/255.
    X_test.append(data)
X_test = np.array(X_test)
Y_test = []
for filepath in os.listdir('../data/test_images/ventral_mask_bulbus_R0004_binary'):
    image = Image.open('../data/test_images/ventral_mask_bulbus_R0004_binary/'+filepath).convert('L')
    image = image.resize((256, 256))
    data = np.asarray(image).reshape(256,256,1)
    data = data/255.
    Y_test.append(data)
Y_test = np.array(Y_test)

In [8]:
from keras.preprocessing.image import ImageDataGenerator

batch_size = 10
epochs = 400
iterations_per_epoch = 300

data_gen_args = dict(rotation_range=0.3,
                    width_shift_range=0.05,
                    height_shift_range=0.05,
                    shear_range=0.05,
                    zoom_range=0.1,
                    horizontal_flip=True,
                    fill_mode='nearest')

train_generator = trainGenerator(batch_size,'../data/train_images','ventral_samples','ventral_mask_bulbus_binary',data_gen_args,image_color_mode='rgb')
val_datagen = ImageDataGenerator()

In [9]:
model = unet(input_size = (256,256,3))
model_checkpoint = ModelCheckpoint('bulbus.hdf5', monitor='val_loss', verbose=1, save_best_only=True)
early_stopping = EarlyStopping(monitor="val_loss",verbose = 1,mode='min',patience=70)
reduce_lr =  ReduceLROnPlateau(monitor = "val_loss", factor = 0.5, patience = 10,verbose = 0, mode = "auto", epsilon = 1e-04, cooldown = 0,min_lr = 1e-5)
model.fit_generator(train_generator,steps_per_epoch=iterations_per_epoch,epochs=epochs,callbacks=[model_checkpoint,early_stopping,reduce_lr],validation_data=val_datagen.flow(X_test, Y_test, batch_size=batch_size))

Instructions for updating:
Please use Model.fit, which supports generators.
Found 557 images belonging to 1 classes.
Found 557 images belonging to 1 classes.
Epoch 1/400
300/300 [==============================] - ETA: 0s - loss: 0.9397 - dice_coef: 0.0603 - accuracy: 0.7487
Epoch 00001: val_loss improved from inf to 0.94171, saving model to bulbus.hdf5
300/300 [==============================] - 149s 496ms/step - loss: 0.9397 - dice_coef: 0.0603 - accuracy: 0.7487 - val_loss: 0.9417 - val_dice_coef: 0.0581 - val_accuracy: 0.9478
Epoch 2/400
300/300 [==============================] - ETA: 0s - loss: 0.9313 - dice_coef: 0.0687 - accuracy: 0.9561
Epoch 00002: val_loss did not improve from 0.94171
300/300 [==============================] - 137s 458ms/step - loss: 0.9313 - dice_coef: 0.0687 - accuracy: 0.9561 - val_loss: 0.9424 - val_dice_coef: 0.0576 - val_accuracy: 0.9738
Epoch 3/400
300/300 [==============================] - ETA: 0s - loss: 0.9302 - dice_coef: 0.0699 - accuracy: 0.9706
Ep

300/300 [==============================] - ETA: 0s - loss: 0.9270 - dice_coef: 0.0731 - accuracy: 0.9900
Epoch 00021: val_loss improved from 0.93878 to 0.93839, saving model to bulbus.hdf5
300/300 [==============================] - 137s 455ms/step - loss: 0.9270 - dice_coef: 0.0731 - accuracy: 0.9900 - val_loss: 0.9384 - val_dice_coef: 0.0615 - val_accuracy: 0.9798
Epoch 22/400
300/300 [==============================] - ETA: 0s - loss: 0.9266 - dice_coef: 0.0734 - accuracy: 0.9900
Epoch 00022: val_loss did not improve from 0.93839
300/300 [==============================] - 136s 453ms/step - loss: 0.9266 - dice_coef: 0.0734 - accuracy: 0.9900 - val_loss: 0.9385 - val_dice_coef: 0.0614 - val_accuracy: 0.9863
Epoch 23/400
300/300 [==============================] - ETA: 0s - loss: 0.9265 - dice_coef: 0.0735 - accuracy: 0.9900
Epoch 00023: val_loss did not improve from 0.93839
300/300 [==============================] - 136s 453ms/step - loss: 0.9265 - dice_coef: 0.0735 - accuracy: 0.9900 - 

300/300 [==============================] - 137s 455ms/step - loss: 0.9243 - dice_coef: 0.0758 - accuracy: 0.9917 - val_loss: 0.9364 - val_dice_coef: 0.0640 - val_accuracy: 0.9869
Epoch 44/400
300/300 [==============================] - ETA: 0s - loss: 0.9243 - dice_coef: 0.0757 - accuracy: 0.9916
Epoch 00044: val_loss improved from 0.93640 to 0.93637, saving model to bulbus.hdf5
300/300 [==============================] - 137s 455ms/step - loss: 0.9243 - dice_coef: 0.0757 - accuracy: 0.9916 - val_loss: 0.9364 - val_dice_coef: 0.0635 - val_accuracy: 0.9880
Epoch 45/400
300/300 [==============================] - ETA: 0s - loss: 0.9237 - dice_coef: 0.0763 - accuracy: 0.9915
Epoch 00045: val_loss improved from 0.93637 to 0.93617, saving model to bulbus.hdf5
300/300 [==============================] - 137s 455ms/step - loss: 0.9237 - dice_coef: 0.0763 - accuracy: 0.9915 - val_loss: 0.9362 - val_dice_coef: 0.0636 - val_accuracy: 0.9856
Epoch 46/400
300/300 [==============================] - ETA

Epoch 66/400
300/300 [==============================] - ETA: 0s - loss: 0.9214 - dice_coef: 0.0785 - accuracy: 0.9922
Epoch 00066: val_loss improved from 0.93467 to 0.93449, saving model to bulbus.hdf5
300/300 [==============================] - 137s 456ms/step - loss: 0.9214 - dice_coef: 0.0785 - accuracy: 0.9922 - val_loss: 0.9345 - val_dice_coef: 0.0660 - val_accuracy: 0.9886
Epoch 67/400
300/300 [==============================] - ETA: 0s - loss: 0.9210 - dice_coef: 0.0790 - accuracy: 0.9923
Epoch 00067: val_loss did not improve from 0.93449
300/300 [==============================] - 136s 453ms/step - loss: 0.9210 - dice_coef: 0.0790 - accuracy: 0.9923 - val_loss: 0.9346 - val_dice_coef: 0.0654 - val_accuracy: 0.9880
Epoch 68/400
300/300 [==============================] - ETA: 0s - loss: 0.9211 - dice_coef: 0.0788 - accuracy: 0.9922
Epoch 00068: val_loss improved from 0.93449 to 0.93429, saving model to bulbus.hdf5
300/300 [==============================] - 137s 455ms/step - loss: 0.

300/300 [==============================] - ETA: 0s - loss: 0.9185 - dice_coef: 0.0816 - accuracy: 0.9927
Epoch 00088: val_loss did not improve from 0.93235
300/300 [==============================] - 136s 453ms/step - loss: 0.9185 - dice_coef: 0.0816 - accuracy: 0.9927 - val_loss: 0.9325 - val_dice_coef: 0.0677 - val_accuracy: 0.9888
Epoch 89/400
300/300 [==============================] - ETA: 0s - loss: 0.9185 - dice_coef: 0.0815 - accuracy: 0.9927
Epoch 00089: val_loss improved from 0.93235 to 0.93228, saving model to bulbus.hdf5
300/300 [==============================] - 137s 455ms/step - loss: 0.9185 - dice_coef: 0.0815 - accuracy: 0.9927 - val_loss: 0.9323 - val_dice_coef: 0.0678 - val_accuracy: 0.9880
Epoch 90/400
300/300 [==============================] - ETA: 0s - loss: 0.9185 - dice_coef: 0.0815 - accuracy: 0.9928
Epoch 00090: val_loss improved from 0.93228 to 0.93207, saving model to bulbus.hdf5
300/300 [==============================] - 137s 455ms/step - loss: 0.9185 - dice_c

300/300 [==============================] - ETA: 0s - loss: 0.9158 - dice_coef: 0.0842 - accuracy: 0.9932
Epoch 00110: val_loss improved from 0.93015 to 0.92955, saving model to bulbus.hdf5
300/300 [==============================] - 137s 455ms/step - loss: 0.9158 - dice_coef: 0.0842 - accuracy: 0.9932 - val_loss: 0.9296 - val_dice_coef: 0.0703 - val_accuracy: 0.9882
Epoch 111/400
300/300 [==============================] - ETA: 0s - loss: 0.9157 - dice_coef: 0.0843 - accuracy: 0.9933
Epoch 00111: val_loss did not improve from 0.92955
300/300 [==============================] - 136s 453ms/step - loss: 0.9157 - dice_coef: 0.0843 - accuracy: 0.9933 - val_loss: 0.9297 - val_dice_coef: 0.0702 - val_accuracy: 0.9892
Epoch 112/400
300/300 [==============================] - ETA: 0s - loss: 0.9155 - dice_coef: 0.0846 - accuracy: 0.9932
Epoch 00112: val_loss did not improve from 0.92955
300/300 [==============================] - 136s 453ms/step - loss: 0.9155 - dice_coef: 0.0846 - accuracy: 0.9932 

300/300 [==============================] - ETA: 0s - loss: 0.9129 - dice_coef: 0.0871 - accuracy: 0.9936
Epoch 00132: val_loss did not improve from 0.92725
300/300 [==============================] - 136s 453ms/step - loss: 0.9129 - dice_coef: 0.0871 - accuracy: 0.9936 - val_loss: 0.9273 - val_dice_coef: 0.0727 - val_accuracy: 0.9891
Epoch 133/400
300/300 [==============================] - ETA: 0s - loss: 0.9126 - dice_coef: 0.0874 - accuracy: 0.9936
Epoch 00133: val_loss did not improve from 0.92725
300/300 [==============================] - 136s 453ms/step - loss: 0.9126 - dice_coef: 0.0874 - accuracy: 0.9936 - val_loss: 0.9273 - val_dice_coef: 0.0727 - val_accuracy: 0.9893
Epoch 134/400
300/300 [==============================] - ETA: 0s - loss: 0.9123 - dice_coef: 0.0877 - accuracy: 0.9936
Epoch 00134: val_loss improved from 0.92725 to 0.92683, saving model to bulbus.hdf5
300/300 [==============================] - 137s 455ms/step - loss: 0.9123 - dice_coef: 0.0877 - accuracy: 0.9936 

300/300 [==============================] - ETA: 0s - loss: 0.9095 - dice_coef: 0.0905 - accuracy: 0.9939
Epoch 00154: val_loss improved from 0.92502 to 0.92497, saving model to bulbus.hdf5
300/300 [==============================] - 137s 455ms/step - loss: 0.9095 - dice_coef: 0.0905 - accuracy: 0.9939 - val_loss: 0.9250 - val_dice_coef: 0.0749 - val_accuracy: 0.9888
Epoch 155/400
300/300 [==============================] - ETA: 0s - loss: 0.9093 - dice_coef: 0.0907 - accuracy: 0.9939
Epoch 00155: val_loss improved from 0.92497 to 0.92446, saving model to bulbus.hdf5
300/300 [==============================] - 137s 455ms/step - loss: 0.9093 - dice_coef: 0.0907 - accuracy: 0.9939 - val_loss: 0.9245 - val_dice_coef: 0.0756 - val_accuracy: 0.9887
Epoch 156/400
300/300 [==============================] - ETA: 0s - loss: 0.9094 - dice_coef: 0.0906 - accuracy: 0.9937
Epoch 00156: val_loss improved from 0.92446 to 0.92418, saving model to bulbus.hdf5
300/300 [==============================] - 137s

300/300 [==============================] - 137s 455ms/step - loss: 0.9061 - dice_coef: 0.0939 - accuracy: 0.9942 - val_loss: 0.9219 - val_dice_coef: 0.0786 - val_accuracy: 0.9892
Epoch 177/400
300/300 [==============================] - ETA: 0s - loss: 0.9059 - dice_coef: 0.0942 - accuracy: 0.9942
Epoch 00177: val_loss improved from 0.92193 to 0.92180, saving model to bulbus.hdf5
300/300 [==============================] - 137s 455ms/step - loss: 0.9059 - dice_coef: 0.0942 - accuracy: 0.9942 - val_loss: 0.9218 - val_dice_coef: 0.0783 - val_accuracy: 0.9882
Epoch 178/400
300/300 [==============================] - ETA: 0s - loss: 0.9060 - dice_coef: 0.0940 - accuracy: 0.9942
Epoch 00178: val_loss did not improve from 0.92180
300/300 [==============================] - 136s 453ms/step - loss: 0.9060 - dice_coef: 0.0940 - accuracy: 0.9942 - val_loss: 0.9219 - val_dice_coef: 0.0780 - val_accuracy: 0.9883
Epoch 179/400
300/300 [==============================] - ETA: 0s - loss: 0.9054 - dice_coe

KeyboardInterrupt: 

In [10]:
import sys
from PIL import Image

# Create side by side png
i=0
for filepath in os.listdir('../data/test_images/ventral_samples_R0004'):
    image = Image.open('../data/test_images/ventral_samples_R0004/'+filepath)
    image = image.resize((256, 256))
    # convert image to numpy array
    data = np.asarray(image)
    data = data/255.
    arr = (model.predict(data.reshape(1,256,256,3))[0]).reshape(256,256)
    mask = Image.fromarray((arr*255))
    new_im = Image.new('RGB', (512, 256))
    new_im.paste(mask, (0,0))
    new_im.paste(image,(256,0))
    new_im.save('vid/%d.png'%(i))
    i=i+1